In [10]:
from model.Data import DataLoader
import numpy as np
import torch
from utils import kld_gauss, unNormalize, load_dataset

In [2]:
train = np.load("./data/reformattedTraffic/train.npz")
val = np.load("./data/reformattedTraffic/val.npz")

In [11]:
batchSize = 10
dataDict = load_dataset("./data/reformattedTraffic/", batchSize)

train
val
test


In [18]:
def getEpochLoss(model, dataLoader, args, dataDict):
    epoch = 0
    totalLoss = 0.0
    for batch_idx, (data, target, dataT, targetT) in enumerate(dataLoader):
        data = torch.as_tensor(data, dtype=torch.float, device="cpu").transpose(0,1)
        target = torch.as_tensor(target, dtype=torch.float, device="cpu").transpose(0,1)
        output = model(data, target, epoch)
        del data
        encoder_means, encoder_stds, decoder_means, decoder_stds, prior_means, prior_stds, all_samples = output
        # Calculate KLDivergence part
        totalKLDLoss = 0.0
        for enc_mean_t, enc_std_t, decoder_mean_t, decoder_std_t, prior_mean_t, prior_std_t, sample in\
        zip(encoder_means, encoder_stds, decoder_means, decoder_stds, prior_means, prior_stds, all_samples):
            kldLoss = kld_gauss(enc_mean_t, enc_std_t, prior_mean_t, prior_std_t)
            totalKLDLoss += args["kld_weight"] * kldLoss
        #Calculate Prediction Loss
        pred = torch.cat([torch.unsqueeze(y, dim=0) for y in all_samples])
        unNPred = unNormalize(pred.detach(), dataDict["train_mean"], dataDict["train_std"])
        unNTarget = unNormalize(target.detach(), dataDict["train_mean"], dataDict["train_std"])
        assert pred.size() == target.size()
        if args["criterion"] == "RMSE":
            predLoss = torch.sqrt(torch.mean((pred - target)**2))    
            unNormalizedLoss = torch.sqrt(torch.mean((unNPred - unNTarget)**2))
        elif args["criterion"] == "L1Loss":
            predLoss = torch.mean(torch.abs(pred - target))
            unNormalizedLoss = torch.mean(torch.abs(unNPred - unNTarget))
        totalLoss += ((totalKLDLoss / args["sequence_len"]) + predLoss)
    return totalLoss

In [8]:
model = torch.load("./save/server2Models/vrnn/vrnn15/vrnn_full_model.pth", map_location='cpu')

In [12]:
args = {"kld_weight": 0.1, "criterion": "RMSE", "sequence_len": 12}

In [19]:
getEpochLoss(model, dataDict["train_loader"].get_iterator(), args, dataDict)

RuntimeError: Cannot initialize CUDA without ATen_cuda library. PyTorch splits its backend into two shared libraries: a CPU library and a CUDA library; this error has occurred because you are trying to use some CUDA functionality, but the CUDA library has not been loaded by the dynamic linker for some reason.  The CUDA library MUST be loaded, EVEN IF you don't directly use any symbols from the CUDA library! One common culprit is a lack of -Wl,--no-as-needed in your link arguments; many dynamic linkers will delete dynamic library dependencies if you don't depend on any of their symbols.  You can check if this has occurred by using ldd on your binary to see if there is a dependency on *_cuda.so library.